In [2]:
!pip install pymilvus sentence_transformers

  Using cached grpcio-1.67.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.9 kB)
  Using cached ujson-5.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.3 kB)
  Using cached milvus_lite-2.4.12-py3-none-macosx_11_0_arm64.whl.metadata (10.0 kB)
Using cached grpcio-1.67.1-cp311-cp311-macosx_10_9_universal2.whl (11.0 MB)
Using cached milvus_lite-2.4.12-py3-none-macosx_11_0_arm64.whl (17.4 MB)
Using cached ujson-5.10.0-cp311-cp311-macosx_11_0_arm64.whl (51 kB)
  Attempting uninstall: grpcio0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
    Found existing installation: grpcio 1.71.0━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
    Uninstalling grpcio-1.71.0:0m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
      Successfully uninstalled grpcio-1.71.0━━━━━━━━━━━━━━━━━━ 1/4 [milvus-lite]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pymilvus]3/4 [pymilvus]


In [1]:
from sentence_transformers import SentenceTransformer
import torch

# 디바이스 설정: MPS 사용
device = "mps" if torch.backends.mps.is_available() else "cpu"
# device = "cpu"
# 모델 로드 및 디바이스 이동
model = SentenceTransformer("BAAI/bge-m3")
model.to(device)

/Users/ranifarm/.conda/envs/adaptive-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
from pymilvus import MilvusClient, DataType

# docker-compose에 설정한 19530포트에 연결
milvus_client = MilvusClient(uri="http://localhost:19530")

In [14]:
# 스키마 정의
from pymilvus import CollectionSchema, FieldSchema, DataType

collection_name = "tragfile"

milvus_client.create_collection(
    collection_name=collection_name,
    schema=CollectionSchema(fields=[
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
        FieldSchema(name="file_name", dtype=DataType.VARCHAR, max_length=500),
        FieldSchema(name="content", dtype=DataType.VARCHAR, max_length=10000),
        FieldSchema(name="embed_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
        FieldSchema(name="page_no", dtype=DataType.VARCHAR, max_length=100)
        ]
    )
)

In [1]:
import psycopg2, os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

POSTGRE_NAME = os.getenv("POSTGRE_NAME")
POSTGRE_USER = os.getenv("POSTGRE_USER")
POSTGRE_HOST = os.getenv("POSTGRE_HOST")
POSTGRE_PORT = os.getenv("POSTGRE_PORT")
POSTGRE_PASSWORD = os.getenv("POSTGRE_PASSWORD")

# env value check
print(POSTGRE_NAME, POSTGRE_HOST)

mydb localhost


In [2]:
# 기존에 만들어놓은 DB 테스트
conn = psycopg2.connect(
        dbname=POSTGRE_NAME,
        user=POSTGRE_USER,
        host=POSTGRE_HOST,
        port=POSTGRE_PORT,
        password=POSTGRE_PASSWORD
    )

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()
        cursor.execute("""
                       SELECT tf01.file_nm, tf02.cont, tf02.page_no_chst
                       FROM tragfile0100 AS tf01
                       INNER JOIN tragfile0102 AS tf02
                       ON tf01.file_seq = tf02.file_seq;
                       """)
        
        result = cursor.fetchall()
        print(f"총 결과 행 수: {len(result)}")

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        db_df = pd.DataFrame(result, columns=columns)


except Exception as e:
    print(f"Error: {e}")

finally:
    conn.close()
    print('db 연결 종료!')

db_df

총 결과 행 수: 6066
db 연결 종료!


,file_nm,cont,page_no_chst
0,190319 고용노동부 부당노동행위수사혁신방안 (수사매뉴얼),부당노동행위 수사 체크리스트_지배·개입 / 노조전임자 급여 지원 / 노동조합 운영비...,"69, 70, 71"
1,190319 고용노동부 부당노동행위수사혁신방안 (수사매뉴얼),부당노동행위 수사 체크리스트_지배·개입 / 노조전임자 급여 지원 / 노동조합 운영비...,"71, 72"
2,190319 고용노동부 부당노동행위수사혁신방안 (수사매뉴얼),부당노동행위 수사 체크리스트_지배·개입 / 노조전임자 급여 지원 / 노동조합 운영비...,73
3,190319 고용노동부 부당노동행위수사혁신방안 (수사매뉴얼),부당노동행위 수사 체크리스트_지배·개입 / 노조전임자 급여 지원 / 노동조합 운영비...,"74, 75"
4,190319 고용노동부 부당노동행위수사혁신방안 (수사매뉴얼),부당노동행위 수사 체크리스트_지배·개입 / 노조전임자 급여 지원 / 노동조합 운영비...,"76, 77, 78"
...,...,...,...
6061,개정 통상임금 노사지도 지침(`25.2.6.),통상임금 노사지도 지침\n3 노사지도 지침\n(1) 통상임금에 관한 지도방향\n\n...,21
6062,개정 통상임금 노사지도 지침(`25.2.6.),"Q10 회계연도 기준으로 연차휴가제도를 운영할 경우, 연차유급 휴가미사용수당 산정방...",37
6063,개정 통상임금 노사지도 지침(`25.2.6.),"Q11 고정시간외수당을 지급해 온 경우, 12.19.이후 어떻게 계산해야 하는지?\...",38
6064,개정 통상임금 노사지도 지침(`25.2.6.),Q12 재직 또는 근무일수 조건이 있는 임금에 대해 앞으로 근무한 기간에 비례하여 ...,39


In [ ]:
# 임베딩
import torch
from sentence_transformers import SentenceTransformer

def sentence_to_vector(sentences: list):
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    model = SentenceTransformer("BAAI/bge-m3", device=device)

    embeddings = model.encode(sentences, batch_size=4, show_progress_bar=True)

    print(embeddings.shape)
    print(embeddings)

    return embeddings



embedding_list = sentence_to_vector(list(db_df['cont']))
embedding_list[:5]

/Users/ranifarm/.conda/envs/adaptive-rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches:   9%|▊         | 130/1517 [08:36<1:31:54,  3.98s/it]


KeyboardInterrupt: 

In [ ]:
import torch
import gc
import numpy as np
from sentence_transformers import SentenceTransformer

def sentence_to_vector(sentences: list, batch_size=4):
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    model = SentenceTransformer("BAAI/bge-m3", device=device)
    model.eval()

    embeddings = []
    with torch.no_grad():
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i + batch_size]

            # encode directly as numpy (no GPU tensor retained)
            emb = model.encode(batch, convert_to_tensor=False, show_progress_bar=False)
            embeddings.append(emb)

            # clean up (very important in MPS)
            gc.collect()
            if torch.backends.mps.is_available():
                torch.mps.empty_cache()

    all_embeddings = np.vstack(embeddings)
    print(all_embeddings.shape)
    return all_embeddings


embedding_list = sentence_to_vector(list(db_df['cont']))
embedding_list[:5]